In [ ]:
import json,requests,dateutil
from datetime import datetime,timedelta
import pandas as pd
from tqdm import tqdm
from requests.auth import HTTPBasicAuth
configFile='../Data/CONFIG.json'
inputFile='../Data/input_data.csv'
getConfig=json.loads(open(configFile,'r').read())

In [ ]:
data=pd.read_csv(inputFile)
data.columns=['timeStamp','c8y_Flow']
print (data.shape)
data.head()

In [ ]:
data['timeStamp']=data['timeStamp'].apply(lambda x: dateutil.parser.parse(x))
cDate=datetime.now()
data['dates']=data['timeStamp'].apply(lambda x: x.date())
data['times']=data['timeStamp'].apply(lambda x: x.time())

In [ ]:
newDf=pd.DataFrame(pd.unique(data['dates']),columns=['dates'])
newDf['index']=list(range(1,newDf.shape[0]+1))[::-1]
newDf['newDate']=newDf['index'].apply(lambda x: cDate.date()-timedelta(days=x))
data=pd.merge(data,newDf[['dates','newDate']],on=['dates'],how='left')

In [ ]:
import requests
import json
auth = HTTPBasicAuth(getConfig['c_user'], getConfig['c_pass'])
url = getConfig['c_url']+"/measurement/measurements"
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/vnd.com.nsn.cumulocity.measurement+json'
}


for i in tqdm(range(data.shape[0])):
    values=data.iloc[i]
    payload = json.dumps({
                          "c8y_Flow": {"F": {"unit": "psi","value": values['c8y_Flow']}},
                          "time": str(values['newDate'])+'T'+str(values['times'])+'.000+05:30',
                          "source": {
                            "id": getConfig['c_device_source']
                          },
                          "type": "c8y_Flow"
                        })
    response = requests.post( url,auth=auth, headers=headers, data=payload)
